## Convert the recommender.pb file to tflite file
(For EdgeTPU)

In [1]:
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf
import torch
import tempfile
import pathlib
import tensorflow_datasets as tfds # for custom dataset
%cd ./clothes-detection-yolov5

In [2]:
resized_img_list = []
train_dataset = tfds.load('new_dataset', split = tfds.Split.TRAIN, shuffle_files=True)    
x = []
for i, data in enumerate(train_dataset):
        if(i >=100):
                break
        emb, label = data['embedding'], data['label']        
        x.append(emb.numpy())        
print(len(x))

100


2023-03-06 17:53:37.825744: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 17:53:37.981600: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [3]:
# 양자화를 위해 대표 이미지 생성기를 두어 효과적으로 양자화 되는 것을 tracking 해야 함
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x).batch(1).take(100):
        yield [input_value]

In [4]:
# For test
gen = representative_data_gen


for i, sample in enumerate(gen()):
  if(i==0):
    print([list(s.shape) for s in sample])

[[1, 200]]


In [5]:
# 8bit-integer quantization tflite model

converter = tf.lite.TFLiteConverter.from_saved_model('./weights/recModel')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error (Full Integer 변환을 위해 operation들이 8bit화를 지원하는지 확인해줌)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8 # or tf.int8
converter.inference_output_type = tf.uint8 # or tf.int8

post_quant_tflite_model = converter.convert()

2023-03-06 17:53:38.264975: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-06 17:53:38.264988: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-06 17:53:38.267058: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: ./weights/recModel
2023-03-06 17:53:38.269668: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-03-06 17:53:38.269677: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: ./weights/recModel
2023-03-06 17:53:38.273408: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-03-06 17:53:38.274723: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-03-06 17:53:38.329976: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: ./weights/recModel
2

In [6]:
# float original tflite model convert
converter = tf.lite.TFLiteConverter.from_saved_model('./weights/recModel')
origin_tflite_model = converter.convert()

2023-03-06 17:53:38.844262: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-06 17:53:38.844276: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-06 17:53:38.844381: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: ./weights/recModel
2023-03-06 17:53:38.845869: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-03-06 17:53:38.845877: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: ./weights/recModel
2023-03-06 17:53:38.850414: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-03-06 17:53:38.899079: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: ./weights/recModel
2023-03-06 17:53:38.914412: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 7

In [7]:
# create temp file -> 사이즈 측정 용도
_, origin_tflite_file = tempfile.mkstemp('.tflite')
_, integer_tflite_file = tempfile.mkstemp('.tflite')

with open(origin_tflite_file, 'wb') as f:
    f.write(origin_tflite_model)
    
with open(integer_tflite_file, 'wb') as f:
    f.write(post_quant_tflite_model)
origin_tflite_size = os.path.getsize(origin_tflite_file) / float(2**20)        
integer_tflite_size = os.path.getsize(integer_tflite_file) / float(2**20)        


print(f"Float tflite model: {origin_tflite_size:.03f} MB")
print(f"quantized tflite model: {integer_tflite_size:.03f} MB")

Float tflite model: 0.357 MB
quantized tflite model: 0.093 MB


In [8]:
# 모델 저장
base_path = '/Users/sinjemin/AmbientAI'
tflite_models_dir = pathlib.Path(base_path + "/clothes-detection-yolov5/weights/quantized_model")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"quantized_recModel.tflite"
tflite_model_quant_file.write_bytes(post_quant_tflite_model)
print(tflite_model_quant_file)

/Users/sinjemin/AmbientAI/clothes-detection-yolov5/weights/quantized_model/quantized_recModel.tflite


For Evaluation

In [9]:
# 8bit integer tflite
post_quant_interpreter = tf.lite.Interpreter(model_content=post_quant_tflite_model)
post_quant_interpreter.allocate_tensors()
post_quant_interpreter.get_output_details()[0]

INFO: Initialized TensorFlow Lite runtime.


{'name': 'StatefulPartitionedCall:0',
 'index': 17,
 'shape': array([1, 4], dtype=int32),
 'shape_signature': array([-1,  4], dtype=int32),
 'dtype': numpy.uint8,
 'quantization': (0.00390625, 0),
 'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
  'zero_points': array([0], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [10]:

!edgetpu_compiler --min_runtime_version 13 $"/Users/sinjemin/AmbientAI/clothes-detection-yolov5/weights/quantized_model/quantized_recModel.tflite" -d 

zsh:1: command not found: edgetpu_compiler
